In [122]:
import pyshacl
from rdflib import *

tbox = Namespace('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')

### PATTERN

In [123]:
contract = Graph()
contract.parse("/home/acraf/psr/tfm/Fdatavalidation/DataPlatformLayer/Integration/SDP1_C.ttl", format="ttl")

<Graph identifier=Nbcca0998592a4e20a9b0fbf92ef1f7d9 (<class 'rdflib.graph.Graph'>)>

In [125]:
construct_query = """
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX tb: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#>
PREFIX ab: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#>

CONSTRUCT {
    ?policy tb:hasOP ?operation .
    ?operation a tb:Operation ;
               tb:hasAttribute ?attribute ;
               tb:hasInput ?input ;
               tb:hasOutput ?output .
    ?attribute a tb:Attribute .
    ?input a tb:Dataset .
    ?output a tb:Value .
}
WHERE {
    ?policy a odrl:Privacy ;
            odrl:duty ?duty .
    ?duty odrl:action odrl:anonymize .
    
    ?duty odrl:target ?target .
    ?target tb:attribute ?attribute .

    # Bind the new URIs for the constructed subgraph
    BIND(IRI(CONCAT(STR(?duty), "/operation")) AS ?operation)
    BIND(IRI(CONCAT(STR(?duty), "/input")) AS ?input)
    BIND(IRI(CONCAT(STR(?duty), "/output")) AS ?output)
}
"""

# Execute the SPARQL CONSTRUCT query
new_graph = contract.query(construct_query)

# Create a new graph for the results
result_graph = Graph()

# Add the constructed triples to the result graph
for triple in new_graph:
    result_graph.add(triple)

# Optionally, serialize the modified graph to a file
print(result_graph.serialize(format="turtle"))

In [247]:
def get_graph():
    
    g = Graph()
    g.bind('ab', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#')
    g.bind('tb', 'http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#')
    g.bind('odrl', 'http://www.w3.org/ns/odrl/2/')
    
    return g

In [248]:
# Define the URI for the specific data product
dp = URIRef("http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#UPENN-GBM_clinical_info_v2")

select_query = f"""
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX tb: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#>
PREFIX ab: <http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#>

CONSTRUCT {{
    ?policy tb:hasOP ?operation .
    ?operation a tb:Operation ;
               tb:hasAttribute ?attribute ;
               tb:hasInput ab:Dataset ;
               tb:hasOutput ab:Dataset .
    ?attribute a tb:Attribute .
    ?input a tb:Dataset .
    ?output a tb:Value .
}}
WHERE {{
    <{dp}> tb:hasDC ?dc .
    ?dc tb:hasPolicy ?policy .
    ?policy a odrl:Privacy ;
            odrl:duty ?duty .
    ?duty odrl:action ?action .
    ?duty odrl:target ?target .
    
    FILTER NOT EXISTS {{ ?duty odrl:action ?otherAction . FILTER(?otherAction != ?action) }}
    FILTER NOT EXISTS {{ ?duty odrl:target ?otherTarget . FILTER(?otherTarget != ?target) }}
    
    BIND( ?action AS ?operation)
    BIND( ?target AS ?attribute)
}}
"""

# Execute the SPARQL SELECT query
results = contract.query(select_query)

# Create a new graph for the results
result_graph = get_graph()

# Add the constructed triples to the result graph
for triple in results:
    result_graph.add(triple)

# Print the new triples
for triple in result_graph:
    print(triple)
    
    
    
# Optionally, serialize the modified graph to a file
result_graph.serialize("modified_data.ttl", format="turtle")

(rdflib.term.URIRef('http://www.w3.org/ns/odrl/2/anonymize'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#Operation'))
(rdflib.term.URIRef('http://www.w3.org/ns/odrl/2/anonymize'), rdflib.term.URIRef('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#hasInput'), rdflib.term.URIRef('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#Dataset'))
(rdflib.term.URIRef('http://www.w3.org/ns/odrl/2/anonymize'), rdflib.term.URIRef('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#hasOutput'), rdflib.term.URIRef('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#Dataset'))
(rdflib.term.URIRef('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/abox#Subject'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.semanticweb.org/acraf/ontologies/2024/healthmesh/tbox#Attri

<Graph identifier=Nd27b77dae7c9491e92d287e9f864a80b (<class 'rdflib.graph.Graph'>)>